<a href="https://colab.research.google.com/github/RandomNoob724/ThesisWork/blob/main/NamedEntityRecognitionTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NAMED ENTITY RECOGNITION:

1. The named entities are pre-defined categories chosen according to the use case such as names of people, organizations, places, codes, time notations, monetary values, etc.

1. NER aims to assign a class to each token (usually a single word) in a sequence. Because of this, NER is also referred to as token classification.

In [ ]:
!nvidia-smi

Wed May 12 14:46:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install simpletransformers
!pip install datasets

## Dataset
Creation of the train and test datasets

In [ ]:
import pandas as pd
data = pd.read_csv("fixed_training_data.txt",encoding="utf8", delimiter="\t")
data = data.fillna(method ="ffill")
data.head(30)

,Sentence #,Word,Tag
0,0,I,0
1,0,längden,0
2,0,torde,0
3,0,få,0
4,0,moderna,0
5,0,par,0
6,0,överleva,0
7,0,om,0
8,0,en,0
9,0,part,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

data["labels"] = data["labels"].str.upper()

X = data[["sentence_id","words"]]
Y = data["labels"]

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

print(len(train_data))
print(len(test_data))

train_data.head(30)

90500
22625


,sentence_id,words,labels
25541,1541,fyra,0
109276,6629,att,0
105159,6355,",",0
40681,2506,man,0
25015,1508,delstatsguvernören,0
312,20,omsättningen,0
40307,2480,.,0
83071,5070,auktoritär,0
3,0,få,0
17628,1062,ta,0


## Model Training


In [ ]:
from simpletransformers.ner import NERModel, NERArgs

label = data["labels"].unique().tolist()

args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32
args.do_lower_case = True


In [ ]:
#model = NERModel('bert', 'KB/bert-base-swedish-cased',labels=label,args =args)
#model = NERModel('albert', 'KB/albert-base-swedish-cased-alpha',labels=label,args =args)
#model = NERModel('bert', 'bert-base-multilingual-cased',labels=label,args =args)
model = NERModel('bert', 'af-ai-center/bert-base-swedish-uncased',labels=label,args =args)
#model = NERModel('electra', 'KB/electra-base-swedish-cased-discriminator',labels=label,args =args)

Some weights of the model checkpoint at af-ai-center/bert-base-swedish-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model chec

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)
result, model_outputs, preds_list = model.eval_model(test_data)
result

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MISC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.07966141057139273,
 'f1_score': 0.679987511707774,
 'precision': 0.7453798767967146,
 'recall': 0.6251435132032147}

## Model Training 2
loopad för fler resultat per model


In [ ]:
from simpletransformers.ner import NERModel, NERArgs

label = data["labels"].unique().tolist()

args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

args.do_lower_case = True


In [ ]:
#model = NERModel('bert', 'KB/bert-base-swedish-cased',labels=label,args =args)
#model = NERModel('albert', 'KB/albert-base-swedish-cased-alpha',labels=label,args =args)
#model = NERModel('bert', 'bert-base-multilingual-cased',labels=label,args =args)
model = NERModel('bert', 'af-ai-center/bert-base-swedish-uncased',labels=label,args =args)
#model = NERModel('electra', 'KB/electra-base-swedish-cased-discriminator',labels=label,args =args)

Some weights of the model checkpoint at af-ai-center/bert-base-swedish-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model chec

In [ ]:
resultList = []
for i in range(10):
  model.train_model(train_data,eval_data = test_data,acc=accuracy_score)
  result, model_outputs, preds_list = model.eval_model(test_data)
  resultList.append(result)
resultList

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MISC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


[{'eval_loss': 0.08090532829082066,
  'f1_score': 0.6850588964662121,
  'precision': 0.7446091644204852,
  'recall': 0.6343283582089553},
 {'eval_loss': 0.08484663639860558,
  'f1_score': 0.6980899568699938,
  'precision': 0.7533244680851063,
  'recall': 0.6504018369690011},
 {'eval_loss': 0.09448950162005544,
  'f1_score': 0.6784624771201953,
  'precision': 0.7239583333333334,
  'recall': 0.6383467278989667},
 {'eval_loss': 0.10895202183347848,
  'f1_score': 0.6831050228310503,
  'precision': 0.7271548930654569,
  'recall': 0.6440872560275546},
 {'eval_loss': 0.11283013422840196,
  'f1_score': 0.6712907117008443,
  'precision': 0.7071156289707751,
  'recall': 0.6389207807118255},
 {'eval_loss': 0.11946427390999621,
  'f1_score': 0.6791181873851806,
  'precision': 0.7276902887139107,
  'recall': 0.6366245694603904},
 {'eval_loss': 0.1333271898544028,
  'f1_score': 0.6596002422774075,
  'precision': 0.698076923076923,
  'recall': 0.6251435132032147},
 {'eval_loss': 0.13658470330016184,
